# Building a Conversational Chatbot for Slack using Rasa and Python -Part 1

## Starting Jupyter Notebook with necessary imports

In [108]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')



# Installations
* Rasa NLU
* Rasa Core
* SpaCy Language Model

In [109]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U rasa_core==0.9.6 rasa_nlu[spacy];

  Using cached https://files.pythonhosted.org/packages/72/de/82e762685a991351a9f89d934b1b25fe04d10462cf4017f93227cb4d9058/rasa_core-0.9.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d6/53/31b6b14e124fa916c10e8e58d650cff57ace0027a9e69a0788b1afcc26f4/rasa_nlu-0.14.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cf/db/f6375ee4b604209d88447bffab074f236d5357a4f6fa38901362311ed18d/scikit_learn-0.19.2-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/03/ad/d732a5d9d50bfcd8aeb6e4a266065a8868829388e4e2b529ff689f1fc923/tensorflow-1.8.0-cp36-cp36m-macosx_10_11_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/6e/59/55c32d59b462a9c3fae2ab25f179f91d61617c0215e9f8ba3f4966b2b8b1/pykwalify-1.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f1/51/d1bd383522c12b313eddd7b97b8e7d6cd2a8e

  Using cached https://files.pythonhosted.org/packages/33/70/86c5fec937ea4964184d4d6c4f0b9551564f821e1c3575907639036d9b90/bleach-1.5.0-py2.py3-none-any.whl


rasa-nlu 0.14.6 has requirement scikit-learn~=0.20.2, but you'll have scikit-learn 0.19.2 which is incompatible.
rasa-core 0.9.6 has requirement rasa-nlu~=0.12.0, but you'll have rasa-nlu 0.14.6 which is incompatible.
  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3
  Found existing installation: bleach 3.0.2
    Uninstalling bleach-3.0.2:
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'RECORD'
Consider using the `--user` option or check the permissions.



In [110]:
!{python} -m spacy download en_core_web_md


    Linking successful
    /usr/local/lib/python3.6/site-packages/en_core_web_md -->
    /usr/local/lib/python3.6/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



## Downloading the English Language Model

In [111]:
!{python} -m spacy link en_core_web_md en --force;


    Linking successful
    /usr/local/lib/python3.6/site-packages/en_core_web_md -->
    /usr/local/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [112]:
!pip install rasa_core

  Using cached https://files.pythonhosted.org/packages/7e/90/bfe484adb16cdad078967b2b480517ec6c1180137e26fb4b015bcb090226/scikit_learn-0.20.3-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl


  Found existing installation: scikit-learn 0.19.2
    Uninstalling scikit-learn-0.19.2:
      Successfully uninstalled scikit-learn-0.19.2


# Importing the Installations

In [113]:
import rasa_nlu
import rasa_core
import spacy



# 1. Teaching the bot to understand user inputs using Rasa NLU

## Preparing the NLU Training Data

In [114]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really




## intent:student_work_authorization
- What does the student’s work authorization look like?
- how is student work authorization?
- how does student work authorization look like?
- Student work authorization look?
- what's Student work authorization

## intent:student_work
- When can a student work?
- When can I work?


## intent:student_work_timings
- How many hours per week can a student work?
- How many hours can I work?
- number of hours I can work?
- no. of hours I can work?

## intent:taxes
- Are F-1 students subject to taxes?
- am I subjected to tax ?
- how much tax shoud I pay ?
- should I pay tax ?

## intent:E_verify
- What is E-Verify?
- how to do e-verify?
- What is E Verify?
- how to do e verify?
- What is EVerify?
- how to do everify

##intent: Visa_interview
-What all do I need for Visa Interview?
-what all documents do I need for Visa Interview?
-Documents needed for Visa Interview?
-What all things do I need for Visa Interview?
-What all papaers do I need for Visa Interview?

##intent: travel_date
-When can I travel to the usa?
-How many days prior to my admission can I travel to the USA?

## intent:customs_immigration
-What all do I need for customs and immigration?
-what all documents do I need for customs and immigration?
-Documents needed for customs and immigration?
-What all things do I need for customs and immigration?
-What all papaers do I need for customs and immigration?

##intent:questions_during_visa
-What are the common questions asked during Visa interview?
-What should I prepare for Visa interview?
-What kind of questions are asked during a visa interview?
-Common questions durin Visa interview?
-Questions asked in a Visa interview?

##intent:health_insurance
-How do I pyrchase University heath insurance?
-What steps are involved in purchasing student health insurance?
-Steps to purchase health insurance?
-international Heath Insurance puchase?

##intent:immunization
-What are the steps involved in immunization?
-What all immunization do I need before getting to the USA?
-Kind of immunization required before reaching?
-What is the proof of immunization that needs to be submitted?

##intent:housing
-Where can I find the housing details of SJSU?
-Accomodation in SJSU?
-Accomodation for international students at SJSU?
-Housing for students near SJSU?
-Housing on campus?

##intent:course_registration
-How many courses do I need to register for?
-How many credits are mandatory?
-Number of credits at SJSU?
-SJSU credits for F1 students?

##intent:tution
-Where can I find details about my tution fees?
-Information about tution fees?
-When do I have to pay for tution fees?
-When can I register for the courses?
-Where do I pay my tution?

##intent:tower_ID
-Where can I get my ID?
-Do I need an ID?
-Procedure to get an ID?
-Documents required to get an ID?
-How do I get my student ID?
-Steps involved in getting student ID card?

## intent: inform
- A [dog](group:shibes)
- [dog](group:shibes)
- [bird](group:birds)
- a [cat](group:cats)
- [cat](group:cats)
- a [bird](group:birds)
- of a [dog](group:shibes)
- of a [cat](group:cats)
- a [bird](group:birds), please
- a [dog](group:shibes), please
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [115]:
%ls

config.yml                       stories.md
domain.yml                       stories_backup.md
errors.json                      story_graph.png
models/                          student_assistant_chatbot.ipynb
nlu.md


## Defining the NLU Model Configuration

In [116]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"
- name: "tokenizer_spacy"
- name: "intent_entity_featurizer_regex"
- name: "intent_featurizer_spacy"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_classifier_sklearn"
"""

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


In [131]:
config = """
language: "en"

pipeline:
- name: "tokenizer_whitespace"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
"""

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


## Training the NLU Model.

In [132]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 111 (20 distinct intents)
	- Found intents: 'travel_date', 'questions_during_visa', 'student_work_authorization', 'mood_affirm', 'mood_deny', 'tution', 'E_verify', 'greet', 'goodbye', 'Visa_interview', 'housing', 'student_work_timings', 'immunization', 'taxes', 'student_work', 'customs_immigration', 'health_insurance', 'tower_ID', 'course_registration', 'inform'
	- entity examples: 10 (1 distinct entities)
	- found entities: 'group'

INFO:rasa_nlu.model:Starting to train component tokenizer_whitespace
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_crf
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_synonyms
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_count_

## Evaluating the NLU model on a random text

In [133]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("Where can I get my ID? "))

{
  "intent": {
    "name": "tower_ID",
    "confidence": 0.940646231174469
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "tower_ID",
      "confidence": 0.940646231174469
    },
    {
      "name": "Visa_interview",
      "confidence": 0.13395103812217712
    },
    {
      "name": "travel_date",
      "confidence": 0.1255820095539093
    },
    {
      "name": "student_work_timings",
      "confidence": 0.06320909410715103
    },
    {
      "name": "tution",
      "confidence": 0.06154104322195053
    },
    {
      "name": "immunization",
      "confidence": 0.061004966497421265
    },
    {
      "name": "questions_during_visa",
      "confidence": 0.05169040709733963
    },
    {
      "name": "student_work",
      "confidence": 0.05110524594783783
    },
    {
      "name": "mood_deny",
      "confidence": 0.02431677281856537
    },
    {
      "name": "course_registration",
      "confidence": 0.019902467727661133
    }
  ],
  "text": "Where can I get my ID? 

## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [120]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)

INFO:tensorflow:Restoring parameters from /Users/saching12/Desktop/python codes/student_assistant_chatbot/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt


INFO:tensorflow:Restoring parameters from /Users/saching12/Desktop/python codes/student_assistant_chatbot/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 111 (20 distinct intents)
	- Found intents: 'travel_date', 'questions_during_visa', 'student_work_authorization', 'mood_affirm', 'mood_deny', 'tution', 'E_verify', 'greet', 'goodbye', 'Visa_interview', 'housing', 'student_work_timings', 'immunization', 'taxes', 'student_work', 'customs_immigration', 'health_insurance', 'tower_ID', 'course_registration', 'inform'
	- entity examples: 10 (1 distinct entities)
	- found entities: 'group'

INFO:rasa_nlu.evaluate:Intent evaluation results:
INFO:rasa_nlu.evaluate:Intent Evaluation: Only considering those 111 examples that have a defined intent out of 111 examples
INFO:rasa_nlu.evaluate:F1-Score:  0.990943324276657

{'intent_evaluation': {'predictions': [{'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9464135766029358},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9633802175521851},
   {'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9512230753898621},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9633802175521851},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9317892789840698},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9479368925094604},
   {'text': 'moin',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9441460371017456},
   {'text': 'hey there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9665208458900452},
   {'text': "let's go",
    'intent': 'greet',
    'predicted': 'gre

# 2. Teaching the bot to respond using Rasa Core

## 1. Writing  Stories

In [121]:
stories_md = """


## Entry_greet
* greet
  - utter_greet
  
## say goodbye
* goodbye
  - utter_goodbye
> goodbye_checkpoint

## question_1_Affirm
* student_work_authorization
  - utter_answer_for_work_authorization
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
## question_1_deny
* student_work_authorization
  - utter_answer_for_work_authorization
  - utter_did_that_help
* mood_deny
  - utter_unclear

## question_1_deny_2
* student_work_authorization
  - utter_answer_for_work_authorization
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
  
## question_2_Affirm
* student_work
  - utter_answer_for_student_work
  - utter_did_that_help
* mood_affirm
  - utter_goodbye

## question_2_deny
* student_work
  - utter_answer_for_student_work
  - utter_did_that_help
* mood_deny
  - utter_unclear

## question_2_deny_2
* student_work
  - utter_answer_for_student_work
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye

##timings
* student_work_timings
  - utter_answer_for_work_timings
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
## timings deny
* student_work_timings
  - utter_answer_for_work_timings
  - utter_did_that_help
* mood_deny
  - utter_unclear
    
## timings deny2
* student_work_timings
  - utter_answer_for_work_timings
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye

##taxes
* taxes
  - utter_answer_for_taxes
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
##taxes deny
* taxes
  - utter_answer_for_taxes
  - utter_did_that_help
* mood_deny
  - utter_unclear
  
##taxes deny 2
* taxes
  - utter_answer_for_taxes
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
## verify
* E_verify
  - utter_answer_E_verify
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
## verify deny
* E_verify
  - utter_answer_E_verify
  - utter_did_that_help
* mood_deny
  - utter_unclear
  
## verify deny2
* E_verify
  - utter_answer_E_verify
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye

## Visa_interview
* Visa_interview
  - utter_answer_Visa_interview
  - utter_did_that_help
* mood_affirm
  - utter_goodbye

## Visa_interview_deny
* Visa_interview
  - utter_answer_Visa_interview
  - utter_did_that_help
* mood_deny
  - utter_unclear
  
## Visa_interview deny2
* Visa_interview
  - utter_answer_Visa_interview
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
## travel_date
* travel_date
  - utter_answer_travel_date
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
## travel_date deny
* travel_date
  - utter_answer_travel_date
  - utter_did_that_help
* mood_deny
  - utter_unclear
  
## travel_date deny2
* travel_date
  - utter_answer_travel_date
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye

##customs_immigration
* customs_immigration
  - utter_answer_customs_immigration
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
##customs_immigration deny
* customs_immigration
  - utter_answer_customs_immigration
  - utter_did_that_help
* mood_deny
  - utter_unclear

##customs_immigration deny2
* customs_immigration
  - utter_answer_customs_immigration
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
##questions_during_visa
* questions_during_visa
  - utter_answer_questions_during_visa
  - utter_did_that_help
* mood_affirm
  - utter_goodbye

##questions_during_visa deny
* questions_during_visa
  - utter_answer_questions_during_visa
  - utter_did_that_help
* mood_deny
  - utter_unclear

##questions_during_visa deny2
* questions_during_visa
  - utter_answer_questions_during_visa
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
##health_insurance
* health_insurance
  - utter_answer_health_insurance
  - utter_did_that_help
* mood_affirm
  - utter_goodbye

##health_insurance deny
* health_insurance
  - utter_answer_health_insurance
  - utter_did_that_help
* mood_deny
  - utter_unclear
  
##health_insurance deny2
* health_insurance
  - utter_answer_health_insurance
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
##immunization
* immunization
  - utter_answer_immunization
  - utter_did_that_help
* mood_affirm
  - utter_goodbye

##immunization deny
* immunization
  - utter_answer_immunization
  - utter_did_that_help
* mood_deny
  - utter_unclear

  
##immunization deny2
* immunization
  - utter_answer_immunization
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
    
##housing
* housing
  - utter_answer_housing
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
##housing deny
* housing
  - utter_answer_housing
  - utter_did_that_help
* mood_deny
  - utter_unclear
  
##housing deny2
* housing
  - utter_answer_housing
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
##course_registration
* course_registration
  - utter_answer_course_registration
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
##course_registration deny
* course_registration
  - utter_answer_course_registration
  - utter_did_that_help
* mood_deny
  - utter_unclear

##course_registration deny2
* course_registration
  - utter_answer_course_registration
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
  
##tution
* tution
  - utter_answer_tution
  - utter_did_that_help
* mood_affirm
  - utter_goodbye
  
##tution
* tution
  - utter_answer_tution
  - utter_did_that_help
* mood_deny
  - utter_unclear

  
##tution
* tution
  - utter_answer_tution
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
##tower_ID
* tution
  - utter_answer_tower_ID
  - utter_did_that_help
* mood_affirm
  - utter_goodbye

##tower_ID
* tution
  - utter_answer_tower_ID
  - utter_did_that_help
* mood_deny
  - utter_unclear
  
##tower_ID
* tution
  - utter_answer_tower_ID
  - utter_did_that_help
* mood_deny
  - utter_unclear
* mood_deny
  - utter_okay
  - utter_goodbye
  
  
## fallback
- utter_unclear

"""

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


## 2. Defining a Domain

## Custom Actions

In [122]:
'''from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display

import requests

class ApiAction(Action):
    def name(self):
        return "action_retrieve_image"

    def run(self, dispatcher, tracker, domain):
        
        group = tracker.get_slot('group')
        
        r = requests.get('http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true'.format(group))
        response = r.content.decode()
        response = response.replace('["',"")
        response = response.replace('"]',"")
   
        
        #display(Image(response[0], height=550, width=520))
        dispatcher.utter_message("Here is something to cheer you up: {}".format(response))'''

'from rasa_core.actions import Action\nfrom rasa_core.events import SlotSet\nfrom IPython.core.display import Image, display\n\nimport requests\n\nclass ApiAction(Action):\n    def name(self):\n        return "action_retrieve_image"\n\n    def run(self, dispatcher, tracker, domain):\n        \n        group = tracker.get_slot(\'group\')\n        \n        r = requests.get(\'http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true\'.format(group))\n        response = r.content.decode()\n        response = response.replace(\'["\',"")\n        response = response.replace(\'"]\',"")\n   \n        \n        #display(Image(response[0], height=550, width=520))\n        dispatcher.utter_message("Here is something to cheer you up: {}".format(response))'

##  Visualising the Training Data

## Install these packages
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
!breq install graphviz

!{python} -m pip install pygraphviz;

In [123]:

from IPython.display import Image
from rasa_core.agent import Agent

agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.png", max_history=2)
Image(filename="story_graph.png")

Processed Story Blocks: 100%|██████████| 48/48 [00:00<00:00, 160547.52it/s]


In [134]:

config = """
policies:
  - name: "KerasPolicy"
    epochs: 300
    featurizer:
    - name: MaxHistoryTrackerFeaturizer
      max_history: 2
      state_featurizer:
        - name: BinarySingleStateFeaturizer
  - name: "MemoizationPolicy"
    max_history: 2
  - name: "FallbackPolicy"
    nlu_threshold: 0.5
    core_threshold: 0.2
    fallback_action_name: "utter_unclear"
""" 

%store config > config.yml


Writing 'config' (str) to file 'config.yml'.


In [135]:
!ls


config.yml                      stories.md
domain.yml                      stories_backup.md
errors.json                     story_graph.png
models                          student_assistant_chatbot.ipynb
nlu.md


## Training a Dialogue Model

In [136]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent
from rasa_core import config
# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain.yml', policies=config.load('config.yml'))

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(training_data)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 48/48 [00:00<00:00, 441.93it/s, # trackers=5]
INFO:rasa_core.policies.keras_policy:Fitting model with 168 total samples and a validation split of 0.1


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 2, 52)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                10880     
_________________________________________________________________
dense (Dense)                (None, 30)                990       
_________________________________________________________________
activation (Activation)      (None, 30)                0         
Total params: 11,870
Trainable params: 11,870
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
168/168 [==============================] - 1s 5ms/step - loss: 3.3931 - acc: 0.0357
Epoch 2/300
168/168 [==============================] - 0s 145us/step - loss: 3.3670 - acc: 0.1310
Epoch 3/300
168/168 [==============================] - 0s 136us/step - loss:

168/168 [==============================] - 0s 138us/step - loss: 1.5240 - acc: 0.6726
Epoch 76/300
168/168 [==============================] - 0s 152us/step - loss: 1.4230 - acc: 0.7440
Epoch 77/300
168/168 [==============================] - 0s 156us/step - loss: 1.3448 - acc: 0.7679
Epoch 78/300
168/168 [==============================] - 0s 151us/step - loss: 1.4562 - acc: 0.6905
Epoch 79/300
168/168 [==============================] - 0s 146us/step - loss: 1.3308 - acc: 0.7381
Epoch 80/300
168/168 [==============================] - 0s 136us/step - loss: 1.3155 - acc: 0.7500
Epoch 81/300
168/168 [==============================] - 0s 132us/step - loss: 1.2643 - acc: 0.7798
Epoch 82/300
168/168 [==============================] - 0s 131us/step - loss: 1.2816 - acc: 0.7679
Epoch 83/300
168/168 [==============================] - 0s 130us/step - loss: 1.2312 - acc: 0.7798
Epoch 84/300
168/168 [==============================] - 0s 126us/step - loss: 1.2120 - acc: 0.7679
Epoch 85/300
168/168 [=

168/168 [==============================] - 0s 135us/step - loss: 0.6562 - acc: 0.7798
Epoch 158/300
168/168 [==============================] - 0s 130us/step - loss: 0.5548 - acc: 0.8393
Epoch 159/300
168/168 [==============================] - 0s 130us/step - loss: 0.5720 - acc: 0.8155
Epoch 160/300
168/168 [==============================] - 0s 130us/step - loss: 0.5386 - acc: 0.8214
Epoch 161/300
168/168 [==============================] - 0s 132us/step - loss: 0.5919 - acc: 0.8036
Epoch 162/300
168/168 [==============================] - 0s 130us/step - loss: 0.6490 - acc: 0.7917
Epoch 163/300
168/168 [==============================] - 0s 132us/step - loss: 0.6692 - acc: 0.7917
Epoch 164/300
168/168 [==============================] - 0s 138us/step - loss: 0.6633 - acc: 0.8036
Epoch 165/300
168/168 [==============================] - 0s 131us/step - loss: 0.6671 - acc: 0.7679
Epoch 166/300
168/168 [==============================] - 0s 138us/step - loss: 0.6669 - acc: 0.7976
Epoch 167/300


168/168 [==============================] - 0s 128us/step - loss: 0.5528 - acc: 0.7917
Epoch 240/300
168/168 [==============================] - 0s 128us/step - loss: 0.4373 - acc: 0.8512
Epoch 241/300
168/168 [==============================] - 0s 131us/step - loss: 0.4483 - acc: 0.8333
Epoch 242/300
168/168 [==============================] - 0s 135us/step - loss: 0.4936 - acc: 0.8333
Epoch 243/300
168/168 [==============================] - 0s 135us/step - loss: 0.5258 - acc: 0.8155
Epoch 244/300
168/168 [==============================] - 0s 129us/step - loss: 0.5092 - acc: 0.8155
Epoch 245/300
168/168 [==============================] - 0s 137us/step - loss: 0.6061 - acc: 0.8095
Epoch 246/300
168/168 [==============================] - 0s 131us/step - loss: 0.5718 - acc: 0.7917
Epoch 247/300
168/168 [==============================] - 0s 132us/step - loss: 0.5792 - acc: 0.7857
Epoch 248/300
168/168 [==============================] - 0s 132us/step - loss: 0.5793 - acc: 0.7917
Epoch 249/300


INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
Processed actions: 168it [00:00, 1416.82it/s, # examples=156]
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to '/Users/saching12/Desktop/python codes/student_assistant_chatbot/models/dialogue'


In [ ]:
!ls


# Talk to your Bot

In [137]:
#Starting the Bot

from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

INFO:tensorflow:Restoring parameters from /Users/saching12/Desktop/python codes/student_assistant_chatbot/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt


INFO:tensorflow:Restoring parameters from /Users/saching12/Desktop/python codes/student_assistant_chatbot/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt


In [138]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])
        

Your bot is ready to talk! Type your messages here or send 'stop'
Where can I get my ID?


My bad I din't get you!!! Is there anything else you would like to ask ?.
How do I get my student ID?


My bad I din't get you!!! Is there anything else you would like to ask ?.
How many courses do I need to register for?


All F-1 students must register for a full-time course load every semester. Full time is considered 9 units per semester for graduate students and 12 units per semester for undergraduates.For more information, come to the International Student Advising (ISA) office for advising
Did that help you?
yes


Bye
stop
